In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [2]:
NBA2019_plays = pd.read_csv("nba_playbyplay_2018-2019.csv")

C:\Users\Can Sarioz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (72,90,91) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
list(NBA2019_plays.columns)

['Unnamed: 0',
 'EVENTMSGACTIONTYPE',
 'EVENTMSGTYPE',
 'EVENTNUM',
 'GAME_ID',
 'HOMEDESCRIPTION',
 'NEUTRALDESCRIPTION',
 'PCTIMESTRING',
 'PERIOD',
 'PERSON1TYPE',
 'PERSON2TYPE',
 'PERSON3TYPE',
 'PLAYER1_ID',
 'PLAYER1_NAME',
 'PLAYER1_TEAM_ABBREVIATION',
 'PLAYER1_TEAM_CITY',
 'PLAYER1_TEAM_ID',
 'PLAYER1_TEAM_NICKNAME',
 'PLAYER2_ID',
 'PLAYER2_NAME',
 'PLAYER2_TEAM_ABBREVIATION',
 'PLAYER2_TEAM_CITY',
 'PLAYER2_TEAM_ID',
 'PLAYER2_TEAM_NICKNAME',
 'PLAYER3_ID',
 'PLAYER3_NAME',
 'PLAYER3_TEAM_ABBREVIATION',
 'PLAYER3_TEAM_CITY',
 'PLAYER3_TEAM_ID',
 'PLAYER3_TEAM_NICKNAME',
 'SCORE',
 'SCOREMARGIN',
 'VISITORDESCRIPTION',
 'WCTIMESTRING',
 'HOME_TEAM',
 'AWAY_TEAM',
 'HOME_SCORE',
 'AWAY_SCORE',
 'DIFFERENCE',
 'TIME',
 'TEAM',
 'TYPE',
 'SUB_TYPE',
 'ASSIST_PLAYER_ID',
 'ASSIST_COUNT',
 'BLOCK_PLAYER_ID',
 'BLOCK_COUNT',
 'REBOUND_PLAYER_ID',
 'REBOUND_TEAM',
 'REBOUND_OFFENSIVE_COUNT',
 'REBOUND_DEFENSIVE_COUNT',
 'JUMP_BALL_HOME_PLAYER_ID',
 'JUMP_BALL_AWAY_PLAYER_ID',
 'JUM

## Creating the Added Columns (with sample data to test)

The needed column additions are made in the excel for the 2018-19 NBA play-by-play data.

In [4]:
NBA2019_plays['DIFFERENCE'] = abs(NBA2019_plays['HOME_SCORE'] - NBA2019_plays['AWAY_SCORE'])

In [5]:
short_nba = NBA2019_plays.sample(50)
short_nba.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,3PT_ATT_HOME,3PT_ATT_AWAY,3PT_ATT,3PT_MADE_PRE,3PT_MADE
460831,278,1,1,387,21800703,NaN,NaN,7:56,3,5.0,...,1366.0,1271.0,1212.0,1076.0,571.0,False,True,True,True,True
190835,444,6,2,662,21800131,MISS Murray 1' Driving Layup,NaN,2:00,4,4.0,...,32.0,167.0,193.0,1282.0,939.0,False,False,False,False,NaN
326550,123,2,6,169,21800419,Evans S.FOUL (P2.T3) (J.Goble),NaN,9:17,2,4.0,...,370.0,370.0,370.0,370.0,444.0,False,False,False,False,NaN
540554,294,1,2,423,21800873,NaN,NaN,7:02,3,5.0,...,777.0,1449.0,736.0,1258.0,1508.0,False,False,False,False,NaN
324246,73,0,8,102,21800414,SUB: Koufos FOR Cauley-Stein,NaN,3:49,1,4.0,...,491.0,491.0,491.0,97.0,97.0,False,False,False,False,NaN


In [6]:
short_nba['3PT_ATT_HOME']= short_nba['HOMEDESCRIPTION'].str.contains('3PT')
short_nba['3PT_ATT_AWAY']= short_nba['VISITORDESCRIPTION'].str.contains('3PT')

In [7]:
short_nba['3PT_ATT'] = (short_nba['3PT_ATT_HOME'] | short_nba['3PT_ATT_AWAY'])
short_nba['3PT_MADE'] = (short_nba['3PT_ATT'] & short_nba['SHOT_MADE'])

In [8]:
short_nba = short_nba[['GAME_ID', 'PCTIMESTRING', 'PERIOD', 'DIFFERENCE','HOMEDESCRIPTION', 'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ABBREVIATION', 'HOME_TEAM', 'AWAY_TEAM', 'FREE_THROW_PLAYER_ID', 'FREE_THROW_MADE', 'SHOT_PLAYER_ID', 'SHOT_MADE', '3PT_MADE']]

In [9]:
short_nba

,GAME_ID,PCTIMESTRING,PERIOD,DIFFERENCE,HOMEDESCRIPTION,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER2_ID,PLAYER2_NAME,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,HOME_TEAM,AWAY_TEAM,FREE_THROW_PLAYER_ID,FREE_THROW_MADE,SHOT_PLAYER_ID,SHOT_MADE,3PT_MADE
460831,21800703,7:56,3,4,NaN,203468,CJ McCollum,POR,203081,Damian Lillard,...,0,NaN,NaN,Thunder,Trail Blazers,NaN,NaN,203468.0,True,False
190835,21800131,2:00,4,17,MISS Murray 1' Driving Layup,1627750,Jamal Murray,DEN,0,NaN,...,202327,Ekpe Udoh,UTA,Nuggets,Jazz,NaN,NaN,1627750.0,False,False
326550,21800419,9:17,2,2,Evans S.FOUL (P2.T3) (J.Goble),1628393,Jawun Evans,PHX,203939,Dwight Powell,...,0,NaN,NaN,Mavericks,Suns,NaN,NaN,NaN,NaN,False
540554,21800873,7:02,3,9,NaN,203953,Jabari Parker,WAS,0,NaN,...,0,NaN,NaN,Hornets,Wizards,NaN,NaN,203953.0,False,False
324246,21800414,3:49,1,4,SUB: Koufos FOR Cauley-Stein,1626161,Willie Cauley-Stein,SAC,201585,Kosta Koufos,...,0,NaN,NaN,Timberwolves,Kings,NaN,NaN,NaN,NaN,False
182279,21800113,7:14,4,5,MISS Fultz 17' Jump Shot,1628365,Markelle Fultz,PHI,0,NaN,...,0,NaN,NaN,Clippers,76ers,NaN,NaN,1628365.0,False,False
486486,21800758,0:35,2,7,NaN,1628372,Dennis Smith Jr.,DAL,0,NaN,...,0,NaN,NaN,Mavericks,Knicks,NaN,NaN,1628372.0,False,False
323781,21800413,6:49,1,13,SUB: Korver FOR Ingles,204060,Joe Ingles,UTA,2594,Kyle Korver,...,0,NaN,NaN,Heat,Jazz,NaN,NaN,NaN,NaN,False
36366,21801033,3:25,4,11,CLIPPERS Timeout: Regular (Full 4 Short 0),1610612746,NaN,NaN,0,NaN,...,0,NaN,NaN,Bulls,Clippers,NaN,NaN,NaN,NaN,False
249024,21800254,8:56,1,0,NaN,202685,Jonas Valanciunas,TOR,0,NaN,...,0,NaN,NaN,Hawks,Raptors,NaN,NaN,NaN,NaN,False


## Filtering the Data

In [50]:
nba2019_plays = NBA2019_plays.copy()

In [51]:
nba2019_plays.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,3PT_ATT_HOME,3PT_ATT_AWAY,3PT_ATT,3PT_MADE_PRE,3PT_MADE,MINUTE
0,0,0,12,2,21801052,NaN,NaN,12:00,1,0.0,...,NaN,NaN,NaN,NaN,False,False,False,False,NaN,1200
1,1,0,10,4,21801052,Jump Ball Horford vs. Jokic: Tip to Millsap,NaN,12:00,1,4.0,...,0.0,0.0,0.0,0.0,False,False,False,False,NaN,1200
2,2,1,2,7,21801052,NaN,NaN,11:41,1,5.0,...,19.0,19.0,19.0,19.0,False,True,True,False,False,1141
3,3,0,4,8,21801052,Tatum REBOUND (Off:0 Def:1),NaN,11:39,1,4.0,...,21.0,21.0,21.0,21.0,False,False,False,False,NaN,1139
4,4,1,2,9,21801052,MISS Horford 15' Jump Shot,NaN,11:34,1,4.0,...,26.0,26.0,26.0,26.0,False,False,False,False,NaN,1134


#### Filtering last quarter and overtime(s)

In [52]:
nba2019_plays = nba2019_plays.loc[nba2019_plays['PERIOD'] >= 4]

#### Couldn't figure out how to filter with time

In [53]:
nba2019_plays['MINUTE'] = [100*int(i[0]) + int(i[1]) for i in nba2019_plays['PCTIMESTRING'].str.split(':')]

In [54]:
nba2019_plays = nba2019_plays.loc[nba2019_plays['MINUTE'] <= 500]

#### Filtering the difference

In [55]:
nba2019_plays = nba2019_plays.loc[abs(nba2019_plays['DIFFERENCE']) <= 6]

In [56]:
nba2019_plays.shape

(30298, 118)

In [88]:
NBA2019_plays['MINUTE'] = [100*int(i[0]) + int(i[1]) for i in NBA2019_plays['PCTIMESTRING'].str.split(':')]
non_clutch_plays = NBA2019_plays.loc[NBA2019_plays['PERIOD'] <= 3].append(
    NBA2019_plays.loc[(NBA2019_plays['PERIOD'] >= 4) & (NBA2019_plays['MINUTE'] > 500)]).append(
    NBA2019_plays.loc[(NBA2019_plays['PERIOD'] >= 4) & (NBA2019_plays['MINUTE'] <= 500) & (abs(NBA2019_plays['DIFFERENCE']) > 6)])

In [89]:
non_clutch_plays.shape

(552170, 118)

In [92]:
NBA2019_plays.shape[0] == nba2019_plays.shape[0] + non_clutch_plays.shape[0]

True

### 1 Player Test

In [16]:
jaylen_brown_19plays = nba2019_plays.loc[nba2019_plays['PLAYER1_ID'] == 1627759]
jaylen_brown_19plays.shape

(55, 118)

In [17]:
jaylen_brown_ft = jaylen_brown_19plays['FREE_THROW_MADE'].sum() / jaylen_brown_19plays['FREE_THROW_MADE'].count()

In [18]:
jaylen_brown_fg = jaylen_brown_19plays['SHOT_MADE'].sum() / jaylen_brown_19plays['SHOT_MADE'].count()

In [19]:
jaylen_brown_3pt = jaylen_brown_19plays['3PT_MADE'].sum() / jaylen_brown_19plays['3PT_MADE'].count()

In [20]:
jaylen_brown_fg, jaylen_brown_3pt, jaylen_brown_ft

(0.5333333333333333, 0.42857142857142855, 0.3333333333333333)

In [21]:
jaylen_brown_19plays['SHOT_MADE'].count(),jaylen_brown_19plays['3PT_MADE'].count(), jaylen_brown_19plays['FREE_THROW_MADE'].count()

(15, 7, 3)

In [22]:
jaylen_brown_total_plays = NBA2019_plays.loc[NBA2019_plays['PLAYER1_ID'] == 1627759]
jaylen_brown_ft_total = jaylen_brown_total_plays['FREE_THROW_MADE'].sum() / jaylen_brown_total_plays['FREE_THROW_MADE'].count()
jaylen_brown_fg_total = jaylen_brown_total_plays['SHOT_MADE'].sum() / jaylen_brown_total_plays['SHOT_MADE'].count()
jaylen_brown_3pt_total = jaylen_brown_total_plays['3PT_MADE'].sum() / jaylen_brown_total_plays['3PT_MADE'].count()
jaylen_brown_fg_total, jaylen_brown_total_plays['SHOT_MADE'].count(), jaylen_brown_3pt_total, jaylen_brown_total_plays['3PT_MADE'].count(), jaylen_brown_ft_total, jaylen_brown_total_plays['FREE_THROW_MADE'].count()

(0.46464646464646464, 792, 0.3442028985507246, 276, 0.6584158415841584, 202)

### Functions

In [62]:
nba2019_plays.head(10)

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,3PT_ATT_HOME,3PT_ATT_AWAY,3PT_ATT,3PT_MADE_PRE,3PT_MADE,MINUTE
444,444,47,1,641,21801052,Irving 3PT Turnaround Jump Shot (30 PTS),NaN,1:22,4,4.0,...,1684.0,1557.0,1639.0,2088.0,True,False,True,True,True,122
843,378,58,1,534,21801069,Dunn 8' Turnaround Hook Shot (18 PTS),NaN,4:55,4,4.0,...,833.0,1189.0,1695.0,1743.0,False,False,False,False,NaN,455
844,379,1,2,535,21801069,NaN,NaN,4:37,4,5.0,...,851.0,1207.0,1713.0,1761.0,False,True,True,False,False,437
845,380,0,4,536,21801069,NaN,NaN,4:33,4,5.0,...,855.0,1211.0,1717.0,1765.0,False,False,False,False,NaN,433
846,381,1,2,537,21801069,NaN,NaN,4:28,4,5.0,...,860.0,1216.0,1722.0,1770.0,False,True,True,False,False,428
847,382,0,4,538,21801069,Dunn REBOUND (Off:0 Def:5),NaN,4:25,4,4.0,...,863.0,1219.0,1725.0,1773.0,False,False,False,False,NaN,425
848,383,57,2,539,21801069,MISS Lopez 3' Driving Hook Shot,NaN,4:15,4,4.0,...,873.0,1229.0,1735.0,1783.0,False,False,False,False,NaN,415
849,384,0,4,541,21801069,NaN,NaN,4:12,4,5.0,...,876.0,1232.0,1738.0,1786.0,False,False,False,False,NaN,412
850,385,1,5,542,21801069,Luwawu-Cabarrot STEAL (2 STL),NaN,4:07,4,5.0,...,881.0,1237.0,1743.0,1791.0,False,False,False,False,NaN,407
851,386,1,1,544,21801069,Harrison 24' 3PT Jump Shot (15 PTS) (Dunn 11 AST),NaN,4:01,4,4.0,...,887.0,1243.0,1749.0,1797.0,True,False,True,True,True,401


In [63]:
def name_getter(data, id_number):
    return list(data.loc[data['PLAYER1_ID'] == id_number]['PLAYER1_NAME'].value_counts().index)

In [64]:
def fg_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
def fg_made_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['SHOT_MADE'].sum()
def fg_attempt_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['SHOT_MADE'].count()

In [65]:
def ft_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
def ft_made_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['FREE_THROW_MADE'].sum()
def ft_attempt_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return player_filtered['FREE_THROW_MADE'].count()

In [66]:
#def three_getter(data, id_number):
#    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
#    return sum(player_filtered.loc[player_filtered['3PT_MADE'] == True]['3PT_MADE']) / sum(player_filtered.loc[player_filtered['3PT_ATT'] == True]['3PT_ATT'])
def three_made_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return sum(player_filtered.loc[player_filtered['3PT_MADE'] == True]['3PT_MADE'])
def three_attempt_getter(data, id_number):
    player_filtered = data.loc[data['PLAYER1_ID'] == id_number]
    return sum(player_filtered.loc[player_filtered['3PT_ATT'] == True]['3PT_ATT'])

In [67]:
def year_getter(file):
    pattern = r'.*\-(\d+).*'
    return re.findall(pattern, file)

### Building the Player Dataframe

In [68]:
counts2019 = nba2019_plays.groupby('PLAYER1_NAME').size().to_frame().rename(columns = {0: 'Count'})

In [69]:
sum(counts2019.loc[counts2019['Count']>55]['Count']) / sum(counts2019['Count'])

0.7833060307059316

In [70]:
counts2019.loc[counts2019['Count']>55].shape

(174, 1)

In [71]:
players2019 = nba2019_plays.groupby('PLAYER1_ID')[['PLAYER1_NAME']].agg(pd.unique)

In [72]:
players2019['FG%'] = [fg_getter(nba2019_plays, x) for x in players2019.index]
players2019['FG MADE'] = [fg_made_getter(nba2019_plays, x) for x in players2019.index]
players2019['FG ATTEMPTED'] = [fg_attempt_getter(nba2019_plays, x) for x in players2019.index]
players2019['FT%'] = [ft_getter(nba2019_plays, x) for x in players2019.index]
players2019['FT MADE'] = [ft_made_getter(nba2019_plays, x) for x in players2019.index]
players2019['FT ATTEMPTED'] = [ft_attempt_getter(nba2019_plays, x) for x in players2019.index]
players2019['3PT MADE'] = [three_made_getter(nba2019_plays, x) for x in players2019.index]
players2019['3PT ATTEMPTED'] = [three_attempt_getter(nba2019_plays, x) for x in players2019.index]
players2019['3PT%'] = players2019['3PT MADE'] / players2019['3PT ATTEMPTED']


<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

In [73]:
players2019['TOTAL FG%'] = [fg_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FG MADE'] = [fg_made_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FG ATTEMPTED'] = [fg_attempt_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FT%'] = [ft_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FT MADE'] = [ft_made_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL FT ATTEMPTED'] = [ft_attempt_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL 3PT MADE'] = [three_made_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL 3PT ATTEMPTED'] = [three_attempt_getter(NBA2019_plays, x) for x in players2019.index]
players2019['TOTAL 3PT%'] = players2019['TOTAL 3PT MADE'] / players2019['TOTAL 3PT ATTEMPTED']

<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

In [74]:
players2019 = players2019.loc[players2019['PLAYER1_NAME'].isin(list(counts2019.loc[counts2019['Count']>127].index))]

In [75]:
players2019['FG DIFFERENCE'] = players2019['FG%'] - players2019['TOTAL FG%']
players2019['FT DIFFERENCE'] = players2019['FT%'] - players2019['TOTAL FT%']
players2019['3PT DIFFERENCE'] = players2019['3PT%'] - players2019['TOTAL 3PT%']

In [76]:
players2019['YEAR'] = [year_getter("nba_playbyplay_2018-2019.csv")[0] for x in players2019.index]

In [77]:
players2019 = players2019[['PLAYER1_NAME', 'FG MADE', 'FG ATTEMPTED', 'FG%', 'TOTAL FG MADE', 'TOTAL FG ATTEMPTED', 'TOTAL FG%', 'FG DIFFERENCE','FT MADE', 'FT ATTEMPTED', 'FT%','TOTAL FT MADE', 'TOTAL FT ATTEMPTED', 'TOTAL FT%', 'FT DIFFERENCE', '3PT MADE', '3PT ATTEMPTED', '3PT%', 'TOTAL 3PT MADE', 'TOTAL 3PT ATTEMPTED', 'TOTAL 3PT%', '3PT DIFFERENCE', 'YEAR']]

In [78]:
players2019.head()

,PLAYER1_NAME,FG MADE,FG ATTEMPTED,FG%,TOTAL FG MADE,TOTAL FG ATTEMPTED,TOTAL FG%,FG DIFFERENCE,FT MADE,FT ATTEMPTED,...,TOTAL FT%,FT DIFFERENCE,3PT MADE,3PT ATTEMPTED,3PT%,TOTAL 3PT MADE,TOTAL 3PT ATTEMPTED,TOTAL 3PT%,3PT DIFFERENCE,YEAR
PLAYER1_ID,,,,,,,,,,,,,,,,,,,,,
2544,LeBron James,37,86,0.430233,558,1095,0.509589,-0.079356,39,55,...,0.665072,0.044019,12,38,0.315789,111,327,0.339450,-0.023660,2019
2548,Dwyane Wade,32,88,0.363636,416,960,0.433333,-0.069697,10,14,...,0.708155,0.006131,7,32,0.218750,86,261,0.329502,-0.110752,2019
101150,Lou Williams,41,92,0.445652,484,1138,0.425308,0.020345,35,41,...,0.876289,-0.022630,10,29,0.344828,105,291,0.360825,-0.015997,2019
200746,LaMarcus Aldridge,31,60,0.516667,684,1319,0.518575,-0.001908,36,44,...,0.847087,-0.028906,0,1,0.000000,10,42,0.238095,-0.238095,2019
200755,JJ Redick,24,65,0.369231,452,1027,0.440117,-0.070886,17,21,...,0.894118,-0.084594,13,42,0.309524,240,605,0.396694,-0.087170,2019


In [79]:
players2019[['FG DIFFERENCE', 'FT DIFFERENCE','3PT DIFFERENCE']].sum(axis=0, skipna=True)

FG DIFFERENCE    -1.433693
FT DIFFERENCE    -0.775891
3PT DIFFERENCE   -0.809358
dtype: float64

In [80]:
players2019.shape, players2019.loc[players2019['FG DIFFERENCE'] >= 0].shape, players2019.loc[players2019['FT DIFFERENCE'] >= 0].shape, players2019.loc[players2019['3PT DIFFERENCE'] >= 0].shape

((57, 23), (17, 23), (22, 23), (16, 23))

### Builder Function

In [93]:
def builder(file, time, difference, min_plays):
    full_data = pd.read_csv(file)
    data = full_data.copy()
    
    full_data['DIFFERENCE'] = abs(full_data['HOME_SCORE'] - full_data['AWAY_SCORE'])
    full_data['3PT_ATT_HOME']= pd.array(full_data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
    full_data['3PT_ATT_AWAY']= pd.array(full_data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
    full_data['3PT_ATT'] = ((full_data['3PT_ATT_HOME']) | (full_data['3PT_ATT_AWAY']))
    full_data['3PT_MADE'] = ((full_data['3PT_ATT']) & (full_data['SHOT_MADE']))
    full_data['MINUTE'] = [100*int(i[0]) + int(i[1]) for i in full_data['PCTIMESTRING'].str.split(':')]
    
    non_clutch = full_data.loc[full_data['PERIOD'] <= 3].append(
    full_data.loc[(full_data['PERIOD'] >= 4) & (full_data['MINUTE'] > time)]).append(
    full_data.loc[(full_data['PERIOD'] >= 4) & (full_data['MINUTE'] <= time) & (abs(full_data['DIFFERENCE']) > difference)])
    
    data['DIFFERENCE'] = abs(data['HOME_SCORE'] - data['AWAY_SCORE'])
    data['3PT_ATT_HOME']= pd.array(data['HOMEDESCRIPTION'].str.contains('3PT'), dtype="boolean")
    data['3PT_ATT_AWAY']= pd.array(data['VISITORDESCRIPTION'].str.contains('3PT'), dtype="boolean")
    data['3PT_ATT'] = ((data['3PT_ATT_HOME']) | (data['3PT_ATT_AWAY']))
    data['3PT_MADE'] = ((data['3PT_ATT']) & (data['SHOT_MADE']))
    data['MINUTE'] = [100*int(i[0]) + int(i[1]) for i in data['PCTIMESTRING'].str.split(':')]
    data = data.loc[data['PERIOD'] >= 4, :]
    data = data.loc[data['MINUTE'] <= time]
    data = data.loc[abs(data['DIFFERENCE']) <= difference, :]
    
    
    counts = data.groupby('PLAYER1_NAME').size().to_frame().rename(columns = {0: 'Count'})
    players = data.groupby(['PLAYER1_ID','PLAYER1_NAME'])['PLAYER1_NAME'].agg(pd.unique).to_frame().droplevel(1)
    players['FG%'] = [fg_getter(data, x) for x in players.index]
    players['FG MADE'] = [fg_made_getter(data, x) for x in players.index]
    players['FG ATTEMPTED'] = [fg_attempt_getter(data, x) for x in players.index]
    players['FT%'] = [ft_getter(data, x) for x in players.index]
    players['FT MADE'] = [ft_made_getter(data, x) for x in players.index]
    players['FT ATTEMPTED'] = [ft_attempt_getter(data, x) for x in players.index]
    players['3PT MADE'] = [three_made_getter(data, x) for x in players.index]
    players['3PT ATTEMPTED'] = [three_attempt_getter(data, x) for x in players.index]
    players['3PT%'] = players['3PT MADE'] / players['3PT ATTEMPTED']
    
    players['NON-CLUTCH FG%'] = [fg_getter(non_clutch, x) for x in players.index]
    players['NON-CLUTCH FG MADE'] = [fg_made_getter(non_clutch, x) for x in players.index]
    players['NON-CLUTCH FG ATTEMPTED'] = [fg_attempt_getter(non_clutch, x) for x in players.index]
    players['NON-CLUTCH FT%'] = [ft_getter(non_clutch, x) for x in players.index]
    players['NON-CLUTCH FT MADE'] = [ft_made_getter(non_clutch, x) for x in players.index]
    players['NON-CLUTCH FT ATTEMPTED'] = [ft_attempt_getter(non_clutch, x) for x in players.index]
    players['NON-CLUTCH 3PT MADE'] = [three_made_getter(non_clutch, x) for x in players.index]
    players['NON-CLUTCH 3PT ATTEMPTED'] = [three_attempt_getter(non_clutch, x) for x in players.index]
    players['NON-CLUTCH 3PT%'] = players['NON-CLUTCH 3PT MADE'] / players['NON-CLUTCH 3PT ATTEMPTED']
    
    players = players.loc[players['PLAYER1_NAME'].isin(list(counts.loc[counts['Count']>min_plays].index))]
    
    players['FG DIFFERENCE'] = players['FG%'] - players['NON-CLUTCH FG%']
    players['FT DIFFERENCE'] = players['FT%'] - players['NON-CLUTCH FT%']
    players['3PT DIFFERENCE'] = players['3PT%'] - players['NON-CLUTCH 3PT%']
    
    players['YEAR'] = [year_getter(file)[0] for x in players.index]
    
    players = players[['PLAYER1_NAME', 'FG MADE', 'FG ATTEMPTED', 'FG%', 'NON-CLUTCH FG MADE', 'NON-CLUTCH FG ATTEMPTED', 'NON-CLUTCH FG%', 'FG DIFFERENCE','FT MADE', 'FT ATTEMPTED', 'FT%','NON-CLUTCH FT MADE', 'NON-CLUTCH FT ATTEMPTED', 'NON-CLUTCH FT%', 'FT DIFFERENCE', '3PT MADE', '3PT ATTEMPTED', '3PT%', 'NON-CLUTCH 3PT MADE', 'NON-CLUTCH 3PT ATTEMPTED', 'NON-CLUTCH 3PT%', '3PT DIFFERENCE', 'YEAR']]
    
    return players

In [94]:
test_players2019 = builder("nba_playbyplay_2018-2019.csv", 500, 6, 127)

C:\Users\Can Sarioz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (72,90,91) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarnin

<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['SHOT_MADE'].sum() / player_filtered['SHOT_MADE'].count()
<ipython-input-64-b6a8ce7f6105>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['S

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid value encountered in long_scalars
  return player_filtered['FREE_THROW_MADE'].sum() / player_filtered['FREE_THROW_MADE'].count()
<ipython-input-65-affb316c20d6>:3: RuntimeWarning: invalid v

In [95]:
test_players2019.shape, players2019.shape

((57, 23), (57, 23))

In [84]:
players2019.equals(test_players2019)

True

In [96]:
test_players2019.head()

,PLAYER1_NAME,FG MADE,FG ATTEMPTED,FG%,NON-CLUTCH FG MADE,NON-CLUTCH FG ATTEMPTED,NON-CLUTCH FG%,FG DIFFERENCE,FT MADE,FT ATTEMPTED,...,NON-CLUTCH FT%,FT DIFFERENCE,3PT MADE,3PT ATTEMPTED,3PT%,NON-CLUTCH 3PT MADE,NON-CLUTCH 3PT ATTEMPTED,NON-CLUTCH 3PT%,3PT DIFFERENCE,YEAR
PLAYER1_ID,,,,,,,,,,,,,,,,,,,,,
2544,LeBron James,37,86,0.430233,521,1009,0.516353,-0.086120,39,55,...,0.658402,0.050689,12,38,0.315789,99,289,0.342561,-0.026771,2019
2548,Dwyane Wade,32,88,0.363636,384,872,0.440367,-0.076731,10,14,...,0.707763,0.006523,7,32,0.218750,79,229,0.344978,-0.126228,2019
101150,Lou Williams,41,92,0.445652,443,1046,0.423518,0.022134,35,41,...,0.878378,-0.024720,10,29,0.344828,95,262,0.362595,-0.017768,2019
200746,LaMarcus Aldridge,31,60,0.516667,653,1259,0.518666,-0.001999,36,44,...,0.850543,-0.032362,0,1,0.000000,10,41,0.243902,-0.243902,2019
200755,JJ Redick,24,65,0.369231,428,962,0.444906,-0.075676,17,21,...,0.901709,-0.092186,13,42,0.309524,227,563,0.403197,-0.093673,2019


In [73]:
players2019.head()

,PLAYER1_NAME,FG MADE,FG ATTEMPTED,FG%,TOTAL FG MADE,TOTAL FG ATTEMPTED,TOTAL FG%,FG DIFFERENCE,FT MADE,FT ATTEMPTED,...,TOTAL FT%,FT DIFFERENCE,3PT MADE,3PT ATTEMPTED,3PT%,TOTAL 3PT MADE,TOTAL 3PT ATTEMPTED,TOTAL 3PT%,3PT DIFFERENCE,YEAR
PLAYER1_ID,,,,,,,,,,,,,,,,,,,,,
2544,LeBron James,37,86,0.430233,558,1095,0.509589,-0.079356,39,55,...,0.665072,0.044019,12,38,0.315789,111,327,0.339450,-0.023660,2019
2548,Dwyane Wade,32,88,0.363636,416,960,0.433333,-0.069697,10,14,...,0.708155,0.006131,7,32,0.218750,86,261,0.329502,-0.110752,2019
101150,Lou Williams,41,92,0.445652,484,1138,0.425308,0.020345,33,39,...,0.876289,-0.030135,10,29,0.344828,105,291,0.360825,-0.015997,2019
200746,LaMarcus Aldridge,31,60,0.516667,684,1319,0.518575,-0.001908,36,44,...,0.847087,-0.028906,0,1,0.000000,10,42,0.238095,-0.238095,2019
200755,JJ Redick,24,65,0.369231,452,1027,0.440117,-0.070886,17,21,...,0.894118,-0.084594,13,42,0.309524,240,605,0.396694,-0.087170,2019
